In [8]:
from collections import defaultdict

import expt
import pandas as pd

In [4]:
runs_dir = "/dfs/scratch1/ranjanr/runs/cs336/2024-05-03"
df = expt.scan(runs_dir)

  0%|          | 0/10 [00:00<?, ?it/s]

In [5]:
df

,script_path,runs_dir,dev,lm_size,vocab_size,context_length,d_model,num_layers,num_heads,d_ff,attn_pdrop,residual_pdrop,batch_size,warmup_steps,benchmark_steps,amp,done
1714795781928288391,/lfs/ampere8/0/ranjanr/cs336-a2/cs336-systems/...,/dfs/scratch1/ranjanr/runs/cs336/2024-05-03,0,small,10000,128,768,12,12,3072,0.0,0.0,16,1,5,0,True
1714795790379185206,/lfs/ampere8/0/ranjanr/cs336-a2/cs336-systems/...,/dfs/scratch1/ranjanr/runs/cs336/2024-05-03,0,small,10000,128,768,12,12,3072,0.0,0.0,16,1,5,1,True
1714795796741125722,/lfs/ampere8/0/ranjanr/cs336-a2/cs336-systems/...,/dfs/scratch1/ranjanr/runs/cs336/2024-05-03,0,medium,10000,128,1024,24,16,4096,0.0,0.0,16,1,5,0,True
1714795860377535384,/lfs/ampere8/0/ranjanr/cs336-a2/cs336-systems/...,/dfs/scratch1/ranjanr/runs/cs336/2024-05-03,0,medium,10000,128,1024,24,16,4096,0.0,0.0,16,1,5,1,True
1714795869118342729,/lfs/ampere8/0/ranjanr/cs336-a2/cs336-systems/...,/dfs/scratch1/ranjanr/runs/cs336/2024-05-03,0,large,10000,128,1280,36,20,5120,0.0,0.0,16,1,5,0,True
1714795889761582337,/lfs/ampere8/0/ranjanr/cs336-a2/cs336-systems/...,/dfs/scratch1/ranjanr/runs/cs336/2024-05-03,0,large,10000,128,1280,36,20,5120,0.0,0.0,16,1,5,1,True
1714795911492260796,/lfs/ampere8/0/ranjanr/cs336-a2/cs336-systems/...,/dfs/scratch1/ranjanr/runs/cs336/2024-05-03,0,xl,10000,128,1600,48,25,6400,0.0,0.0,16,1,5,0,True
1714795949038588417,/lfs/ampere8/0/ranjanr/cs336-a2/cs336-systems/...,/dfs/scratch1/ranjanr/runs/cs336/2024-05-03,0,xl,10000,128,1600,48,25,6400,0.0,0.0,16,1,5,1,True
1714795971807577187,/lfs/ampere8/0/ranjanr/cs336-a2/cs336-systems/...,/dfs/scratch1/ranjanr/runs/cs336/2024-05-03,0,2.7b,10000,128,2560,32,32,10240,0.0,0.0,16,1,5,0,True
1714796024046762300,/lfs/ampere8/0/ranjanr/cs336-a2/cs336-systems/...,/dfs/scratch1/ranjanr/runs/cs336/2024-05-03,0,2.7b,10000,128,2560,32,32,10240,0.0,0.0,16,1,5,1,True


In [27]:
def make_pd_tab(pass_):
    tab = defaultdict(dict)
    for lm_size in [
        "small",
        "medium",
        "large",
        "xl",
        "2.7b",
    ]:
        amp_time = {}
        for amp in [
            0,
            1,
        ]:
            ts = df.query("lm_size == @lm_size and amp == @amp").index[-1]
            store = expt.PathDict(f"{runs_dir}/{ts}")
            pass_time = store[f"{pass_}_time"]
            amp_time[amp] = pass_time.mean().item()
            tab[lm_size][f"time (s) with {amp=}"] = f"{amp_time[amp]:.3f}"
        speedup = amp_time[0] / amp_time[1]
        tab[lm_size][f"speedup"] = f"{speedup:.2f}"
    tab_df = pd.DataFrame(tab)
    return tab_df

In [28]:
for pass_ in [
    "fw",
    "bw",
]:
    tab_df = make_pd_tab(pass_)
    print(f"pass={pass_}")
    display(tab_df)

pass=fw


,small,medium,large,xl,2.7b
time (s) with amp=0,0.069,0.216,0.501,0.893,1.451
time (s) with amp=1,0.034,0.076,0.122,0.256,0.278
speedup,2.03,2.86,4.11,3.49,5.21


pass=bw


,small,medium,large,xl,2.7b
time (s) with amp=0,0.134,0.415,1.042,1.810,2.967
time (s) with amp=1,0.040,0.099,0.233,0.459,0.504
speedup,3.37,4.17,4.48,3.94,5.88
